In [168]:
import pandas as pd
import numpy as np

In [169]:
orders_df = pd.read_csv("orders.csv")
nodes_df = pd.read_csv("nodes.csv")

orders_df.running_time = pd.to_datetime(orders_df.running_time, format='%Y-%m-%d %H:%M:%S')
orders_df.completed_time = pd.to_datetime(orders_df.completed_time, format='%Y-%m-%d %H:%M:%S')
nodes_df.dropna(inplace=True)

In [170]:
final_test = pd.read_csv("final_test.csv")
final_nodes = pd.read_csv("nodes_test.csv")

final_test.running_time = pd.to_datetime(final_test.running_time, format='%Y-%m-%d %H:%M:%S')
final_nodes.dropna(inplace=True)

In [171]:
mean_speed_list = []

for i in orders_df["Id"]:
    mean_speed_list.append(nodes_df[nodes_df["Id"] == i].iloc[0, 4].mean())

orders_df["mean_nodes_speed"] = mean_speed_list


def day_period(hours):
    if 3 <= hours < 7:
        return 0
    elif 7 <= hours < 12:
        return 1
    elif 12 <= hours < 18:
        return 2
    else:
        return 3


day_period_list = []
for i in orders_df["Id"]:
    day_period_list.append(day_period(orders_df[orders_df["Id"] == i].iloc[0, 1].hour))

orders_df["day_period"] = day_period_list
orders_df = orders_df.drop(np.where(orders_df.route_distance_km == 0)[0])

In [172]:
mean_speed_list = []

for i in final_test["Id"]:
    mean_speed_list.append(final_nodes[final_nodes["Id"] == i].iloc[0, 4].mean())

final_test["mean_nodes_speed"] = mean_speed_list


def day_period(hours):
    if 3 <= hours < 7:
        return 0
    elif 7 <= hours < 12:
        return 1
    elif 12 <= hours < 18:
        return 2
    else:
        return 3


day_period_list = []
for i in final_test["Id"]:
    day_period_list.append(day_period(final_test[final_test["Id"] == i].iloc[0, 1].hour))

final_test["day_period"] = day_period_list
final_test = final_test.drop(np.where(final_test.route_distance_km == 0)[0])
final_test

,Id,running_time,route_distance_km,mean_nodes_speed,day_period
0,6198,2022-01-24 03:38:30,4.744,37.0,0
1,6417,2022-01-24 03:45:51,6.279,45.0,0
2,7054,2022-01-24 03:52:14,3.934,40.0,0
3,9628,2022-01-24 04:03:21,5.959,43.0,0
4,10283,2022-01-24 04:01:12,7.028,30.0,0
...,...,...,...,...,...
995,525706,2022-01-24 18:46:17,2.897,47.0,3
996,526604,2022-01-24 18:46:44,3.482,17.0,3
997,527213,2022-01-24 18:47:25,3.486,28.0,3
998,527520,2022-01-24 18:52:01,0.703,36.0,3


In [173]:
odesa_df = pd.read_csv("nodes/odesa.csv")
nodes_df["node_start_lat"] = nodes_df["node_start"].map(odesa_df.set_index("id")["lat"]).dropna()
nodes_df["node_start_lon"] = nodes_df["node_start"].map(odesa_df.set_index("id")["lon"]).dropna()
nodes_df.dropna(inplace=True)

final_nodes["node_start_lat"] = final_nodes["node_start"].map(odesa_df.set_index("id")["lat"]).dropna()
final_nodes["node_start_lon"] = final_nodes["node_start"].map(odesa_df.set_index("id")["lon"]).dropna()
final_nodes.dropna(inplace=True)

In [174]:
mean_lat = nodes_df.groupby(by="Id").apply(lambda df: df["node_start_lat"].mean())
mean_lon = nodes_df.groupby(by="Id").apply(lambda df: df["node_start_lon"].mean())
orders_df["node_start_lat"] = orders_df["Id"].map(mean_lat)
orders_df["node_start_lon"] = orders_df["Id"].map(mean_lon)

mean_lat = final_nodes.groupby(by="Id").apply(lambda df: df["node_start_lat"].mean())
mean_lon = final_nodes.groupby(by="Id").apply(lambda df: df["node_start_lon"].mean())
final_test["node_start_lat"] = final_test["Id"].map(mean_lat)
final_test["node_start_lon"] = final_test["Id"].map(mean_lon)

In [175]:
orders_df.dropna(inplace=True)
orders_df["mean_speed"] = orders_df["route_distance_km"] / (orders_df["delta_time"] / 3600)
x_train, y_train = orders_df[orders_df.columns].drop(
    ['Id', 'completed_time', 'delta_time', "mean_speed"], axis=1), orders_df["delta_time"]

final_test.dropna(inplace=True)

In [176]:
x_test = final_test[final_test.columns].drop(
    ['Id'], axis=1)

SEED = 42
from sklearn.metrics import make_scorer, mean_squared_error

mse = make_scorer(mean_squared_error, greater_is_better=False)

from xgboost import XGBRegressor

model_xgb = XGBRegressor(
    random_state=SEED,
    colsample_bytree=0.9,
    learning_rate=0.05,
    max_depth=5,
    n_estimators=150,
    subsample=0.9,
)

In [178]:
test_train_list = []

train_pairs = []
for i in x_test.index:
    xt = x_test.loc[i]

    x_tr = x_train[x_train["running_time"] < xt["running_time"]].drop(["running_time"], axis=1)
    y_tr = y_train[y_train.index.isin(x_tr.index)]

    train_pairs.append((x_tr, y_tr, xt))

len(train_pairs)

1000

In [ ]:
preds = []
for pair in train_pairs:
    x_tr, y_tr, xt = pair
    model = model_xgb.fit(x_tr.astype("float"), y_tr.astype("float"))
    prediction = model.predict(xt.to_frame().T.drop(["running_time"], axis=1).astype("float"))
    print(prediction)
    preds.append(prediction)

preds_np = np.array(preds)

In [184]:
ids = final_test.drop(
    columns=["running_time", "route_distance_km", "mean_nodes_speed", "day_period", "node_start_lat", "node_start_lon"])
ids

,Id
0,6198
1,6417
2,7054
3,9628
4,10283
...,...
995,525706
996,526604
997,527213
998,527520


In [185]:
ids["Predicted"] = preds_np

In [188]:
ids.to_csv("final/final_test.csv", index=False)